In [2]:
import transformers
import torch
import math
import pandas as pd
import numpy as np
import emoji
import re
from transformers import (
    RobertaForSequenceClassification, RobertaTokenizer, BertForSequenceClassification, 
    BertTokenizer, AutoModelForSequenceClassification, AutoTokenizer, AdamW
)
import random
import time


seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


/Users/setone/miniforge3/envs/tf-gpu/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# from dev_test import *
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [7]:
pd.read_csv('./data/scraped_recession.csv')

FileNotFoundError: [Errno 2] No such file or directory: './data/recession.csv'

In [6]:
# new
def update_line_chart(company, yrs):
    # if analysis == [] or company == 'All':
    #     return {}, []

#     if company == 'All':
#         # change this when you're done with testing
#         df = globals()['AAPL'][globals()['AAPL'].Date.dt.year.between(yrs[0], yrs[1])]
#     else:
#         df = globals()[company][globals()[company].Date.dt.year.between(yrs[0], yrs[1])]

    # data = df.to_dict("records")

    df_comp = combined_df[combined_df['company'] == company]
    df_comp = df_comp.groupby('date', as_index=False)['sentiment'].mean()
    df_comp['SMA30'] = df_comp['sentiment'].rolling(30).mean()
    df_comp['SMA90'] = df_comp ['sentiment'].rolling(90).mean()
    df_comp = df_comp.rename(columns={'date': 'Date'})

    stock_comp = stocks_df[stocks_df['Stock Name'] == company]
    stock_comp['Date'] = pd.to_datetime(stock_comp['Date'])
    stock_comp['Date'] = stock_comp['Date'].dt.date
    combined = pd.merge(df_comp, stock_comp, how='left', on="Date")
    #data = combined.to_dict("records")
    # Create figure with secondary y-axis
    fig = make_subplots(
        rows=4, cols=1, specs=[
                                [{"secondary_y": True, 'rowspan': 2}], 
                                [None],
                                [{'rowspan': 1}],
                                [{'rowspan': 1}],
                              ], 
        vertical_spacing=0.1)
    
    sentvol_row = 3
    stockvol_row = 4
    
#         fig = make_subplots(
#         rows=4, cols=1,
#         specs=[[{'rowspan': 2}],
#             [None],
#             [{'rowspan': 1}],
#             [None]],
#         vertical_spacing=0.05)
    

    # Add traces
    fig.add_trace(
        go.Candlestick(x=combined['Date'],
                       open=combined['Open'],
                       high=combined['High'],
                       low=combined['Low'],
                       close=combined['Close'],
                       name="",
                       hoverinfo='all'),
        secondary_y=True, 
    )
    
    combined1 = combined.copy()

    # split data into chunks where averages cross each other
    combined['label'] = np.where(combined['SMA30']>combined['SMA90'], 1, 0)
    combined['group'] = combined['label'].ne(combined['label'].shift()).cumsum()
    combined2 = combined.groupby('group')
    combined_s = []
    for name, data in combined2:
        combined_s.append(data)

    # custom function to set fill color
    def fillcol(label):
        if label >= 1:
            return 'rgba(0,250,0,0.4)'
        else:
            return 'rgba(250,0,0,0.4)'

    for df in combined_s:
        fig.add_traces(go.Scatter(x=df.Date, y = df.SMA30,
                                  line = dict(color='rgba(0,0,0,0)'), 
                                  hoverinfo='skip'
                                 )
                      )
        
        fig.add_traces(go.Scatter(x=df.Date, y = df.SMA90,
                                line = dict(color='rgba(0,0,0,0)'),
                                fill='tonexty', 
                                fillcolor = fillcol(df['label'].iloc[0]),
                                hoverinfo='skip'
                                 ))

    # include averages
    fig.add_traces(go.Scatter(x=combined1.Date, y = combined1.SMA30,
                              line = dict(color = 'green', width=1), 
                              name='MA30', hoverinfo='skip'
                             ))

    fig.add_traces(go.Scatter(x=combined1.Date, y = combined1.SMA90,
                              line = dict(color = 'red', width=1), 
                              name='MA90', hoverinfo='skip'
                             ))
    df2 = combined_df[combined_df['company'] == company]
    comp_group = df2.groupby(by=["date", "sentiment"], as_index=False).agg(
        count_col=pd.NamedAgg(column="sentiment", aggfunc="count"))
    
    
    # Subplot - Sentiment Volume

    trace1 = go.Bar(
        x = comp_group['date'],
        y = comp_group[comp_group['sentiment'] == 1]['count_col'],
        name='Bullish',
        marker_color='green',
        marker_line_width=0,
    )
    trace2 = go.Bar(
        x = comp_group['date'],
        y = comp_group[comp_group['sentiment'] == -1]['count_col'],
        name='Bearish',
        marker_color='red',
        marker_line_width=0,
    )
    fig.add_traces([trace1, trace2], rows=sentvol_row, cols=1)
    
    
    fig.update_layout(barmode = 'stack')

    stock_vol = go.Bar(
        x = combined['Date'],
        y = combined['Volume'],
        name = 'Volume',
        marker_color='blue',
    )
    fig.add_trace(stock_vol, row=stockvol_row, col=1)

    # Set title
    fig.layout.update(title=f'{company} Stock Price v. Sentiment',
                     height=600, width=850, 
                      showlegend=True, hovermode='closest')

    # Set x-axis title
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=sentvol_row, col=1)
    fig.update_xaxes(title_text="Date", row=stockvol_row, col=1)
    fig.update_layout(xaxis_rangeslider_visible=False)
    # Set y-axes titles
    fig.update_yaxes(title_text="Stock Price", secondary_y=True, row=1, col=1)
    fig.update_yaxes(title_text="Stock Sentiment", secondary_y=False, row=1, col=1)
    fig.update_yaxes(title_text="Sentiment Volume", secondary_y=False, row=sentvol_row, col=1, range=[0, 1500])
    fig.update_yaxes(title_text="Stock Volume", secondary_y=False, row=stockvol_row, col=1)
    # Set y-axes ticks
    
    fig.update_layout(
    width=1000,
    height=850,
    showlegend=False,
    hovermode='x unified', 
    template='plotly_white',
    legend=dict(
    x=0,
    y=1.05,
    traceorder="normal",
    font=dict(
        family="sans-serif",
        size=12,
        color="black"
    )))
    fig.update_traces(xaxis='x1')
    
    return fig


update_line_chart('AMZN', [2017, 2020])

NameError: name 'combined_df' is not defined

In [81]:
vol_range = {
    ''
}

In [194]:
tokenizer_loaded = RobertaTokenizer.from_pretrained('zhayunduo/roberta-base-stocktwits-finetuned')
model_loaded = RobertaForSequenceClassification.from_pretrained('zhayunduo/roberta-base-stocktwits-finetuned')

In [193]:
checkSenti

NameError: name 'checkSenti' is not defined

In [84]:
if 'Bullish' in checkSenti('im feelin good todays'):
    color = 'success'
else:
    color = 'danger'

In [86]:
[dbc.Button(f"{color}", color=f"{color}", size="sm") for color in theme_colors]

'success'

In [ ]:
[dbc.Button(f"{color}", color=f"{color}", size="sm") for color in theme_colors]

In [60]:
checkSenti('im feeling bearish')

'Sentiment is predicted to be Bearish, 0.9762407648711596'

In [32]:
process_text('craaaaaaap im feeling bullish')

'craaaaaaap im feeling bullish'

In [38]:
checkSenti('im not bullish')

tensor([[ 1.9979, -1.7745]])


'Bearish'

In [37]:
checkSenti('im not bullish')
checkSenti('im feeling bullish')

tensor([[ 1.9979, -1.7745]])
tensor([[-1.9449,  1.8161]])


'Bullish'

In [81]:
samples = ['im feeling bullish about this stock',
           'im feeling bearish about this stock',
           'welp diamond hands it is',
           'shoot to the moon',
           'short this stock']
for sample in samples:
    print(checkSenti(sample))

Sentiment is predicted to be 99.79% Bullish
Sentiment is predicted to be 98.72% Bearish
Sentiment is predicted to be 75.84% Bullish
Sentiment is predicted to be 95.46% Bullish
Sentiment is predicted to be 99.43% Bearish
